### Imports and Setup

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import numpy as np
import random


In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())


2.5.1+cu121
12.1
True


In [3]:
# Set random seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# Determine the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


Using device: cuda


In [4]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

# Initialize the model
base_model = AutoModel.from_pretrained("microsoft/deberta-v3-base")
base_model.to(device)


d:\Documents\FYP\emotion-retrieval-embeddings\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


DebertaV2Model(
  (embeddings): DebertaV2Embeddings(
    (word_embeddings): Embedding(128100, 768, padding_idx=0)
    (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): DebertaV2Encoder(
    (layer): ModuleList(
      (0-11): 12 x DebertaV2Layer(
        (attention): DebertaV2Attention(
          (self): DisentangledSelfAttention(
            (query_proj): Linear(in_features=768, out_features=768, bias=True)
            (key_proj): Linear(in_features=768, out_features=768, bias=True)
            (value_proj): Linear(in_features=768, out_features=768, bias=True)
            (pos_dropout): Dropout(p=0.1, inplace=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): DebertaV2SelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
            (dropout): Dropout(p=0.1, 

### Load Go emotion

In [5]:
# 0. Import
from datasets import load_dataset
from transformers import AutoTokenizer
import torch
from collections import Counter
from torch.utils.data import DataLoader

In [6]:
from datasets import load_dataset
from collections import Counter
import json
import os
from pathlib import Path

# -------------------------------
# 1. Load raw GoEmotions dataset
# -------------------------------
raw_datasets = load_dataset("go_emotions")

# -------------------------------
# 2. Filter to single-label samples (train/validation/test separately)
# -------------------------------
def filter_single_label(split):
    filtered = []
    for example in raw_datasets[split]:
        labels = example["labels"]
        if isinstance(labels, list) and len(labels) == 1:
            filtered.append({"text": example["text"], "label": labels[0]})
    return filtered

filtered_train = filter_single_label("train")
filtered_validation = filter_single_label("validation")
filtered_test = filter_single_label("test")

# For statistics
filtered_train_labels = [sample["label"] for sample in filtered_train]
class_counter = Counter(filtered_train_labels)

# -------------------------------
# 3. Save the three filtered splits as JSONL
# -------------------------------
save_dir = Path("..") / "data" / "splits_single_label"
save_dir.mkdir(parents=True, exist_ok=True)

def save_jsonl(data, name):
    with open(save_dir / f"{name}.jsonl", "w", encoding="utf-8") as f:
        for item in data:
            json.dump(item, f, ensure_ascii=False)
            f.write("\n")

save_jsonl(filtered_train, "train")
save_jsonl(filtered_validation, "validation")
save_jsonl(filtered_test, "test")

# -------------------------------
# 4. Output some confirmation
# -------------------------------
{
    "train_single_label_count": len(filtered_train),
    "validation_single_label_count": len(filtered_validation),
    "test_single_label_count": len(filtered_test),
    "train_class_distribution": dict(sorted(class_counter.items()))
}


{'train_single_label_count': 36308,
 'validation_single_label_count': 4548,
 'test_single_label_count': 4590,
 'train_class_distribution': {0: 2710,
  1: 1652,
  2: 1025,
  3: 1451,
  4: 1873,
  5: 649,
  6: 858,
  7: 1389,
  8: 389,
  9: 709,
  10: 1402,
  11: 498,
  12: 203,
  13: 510,
  14: 430,
  15: 1857,
  16: 39,
  17: 853,
  18: 1427,
  19: 85,
  20: 861,
  21: 51,
  22: 586,
  23: 88,
  24: 353,
  25: 817,
  26: 720,
  27: 12823}}

### Data Agmentation

In [7]:
import math
import os
import torch
from transformers import pipeline
from collections import Counter
from torch.utils.data import Dataset


In [8]:
import os
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# -------------------------------
# 0. Compute correct save path
# -------------------------------
# Go up one level from notebook folder
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
model_save_path = os.path.join(project_root, "models", "bart_paraphraser")

# -------------------------------
# 1. Check if model already exists
# -------------------------------
if os.path.exists(model_save_path) and os.path.isdir(model_save_path):
    print(f"BART paraphraser already exists at {model_save_path}. Skipping download and save.")

else:
    print(f"Model not found at {model_save_path}. Downloading and saving...")

    # -------------------------------
    # 2. Download and save
    # -------------------------------
    model = AutoModelForSeq2SeqLM.from_pretrained("eugenesiow/bart-paraphrase")
    tokenizer = AutoTokenizer.from_pretrained("eugenesiow/bart-paraphrase")

    os.makedirs(model_save_path, exist_ok=True)
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)

    print(f"Model successfully saved to {model_save_path}")


BART paraphraser already exists at d:\Documents\FYP\emotion-retrieval-embeddings\models\bart_paraphraser. Skipping download and save.


In [9]:
from collections import Counter

# Define minority classes
minority_class_ids = [class_id for class_id, count in class_counter.items() if count < 300]

print(f"Minority classes (less than 300 samples): {minority_class_ids}")


Minority classes (less than 300 samples): [12, 23, 21, 16, 19]


#### Full agmentation

In [10]:
import math
from collections import Counter

# -------------------------------
# Step 1: Find minority classes
# -------------------------------

# Assume:
# - `filtered_labels` = list of ground-truth labels after filtering (single-label)
# - `target_samples_per_class` = target number of samples per class

target_samples_per_class = 300
num_classes = 28

# Find classes that are minority (less than target)
minority_classes = [class_id for class_id, count in class_counter.items() if count < target_samples_per_class]

print("Minority Classes to Augment:", minority_classes)

# -------------------------------
# Step 2: Calculate generation goals
# -------------------------------

# For each minority class:
# - How many samples to generate
# - How many paraphrases needed per original sample
class_to_generation_goal = {}
class_to_paraphrase_per_sample = {}

for class_id in minority_classes:
    current_count = class_counter[class_id]
    needed_count = target_samples_per_class - current_count

    # Number of paraphrases needed per sample
    # r_c = ceil(needed / existing)
    paraphrases_per_sample = math.ceil(needed_count / current_count)

    class_to_generation_goal[class_id] = needed_count
    class_to_paraphrase_per_sample[class_id] = paraphrases_per_sample

# -------------------------------
# Step 3: Print planning table
# -------------------------------

print("\nSamples and paraphrases needed per minority class:")
print("{:<10} {:<20} {:<30}".format("Class ID", "Samples to Generate", "Paraphrases per Original Sample"))

for class_id in minority_classes:
    print("{:<10} {:<20} {:<30}".format(
        class_id,
        class_to_generation_goal[class_id],
        class_to_paraphrase_per_sample[class_id]
    ))


Minority Classes to Augment: [12, 23, 21, 16, 19]

Samples and paraphrases needed per minority class:
Class ID   Samples to Generate  Paraphrases per Original Sample
12         97                   1                             
23         212                  3                             
21         249                  5                             
16         261                  7                             
19         215                  3                             


In [11]:
def paraphrase_text(text, num_return_sequences, paraphraser_pipeline):
    """
    Generate paraphrases for a given text.

    Args:
        text (str): The input text to paraphrase.
        num_return_sequences (int): How many paraphrased versions to generate.
        paraphraser_pipeline (transformers.Pipeline): A HuggingFace pipeline for text2text-generation.

    Returns:
        List[str]: A list of generated paraphrased texts.
    """
    try:
        # Generate paraphrases
        outputs = paraphraser_pipeline(
            text,
            num_return_sequences=num_return_sequences,
            num_beams=num_return_sequences + 4,  # +4 beams to increase diversity
            max_length=128,
            do_sample=False  # Deterministic decoding (no randomness)
        )

        # Extract generated texts
        paraphrased_texts = [o['generated_text'] for o in outputs]
        return paraphrased_texts

    except Exception as e:
        print(f"[ERROR] Paraphrasing failed: {e}")
        return []  # Return empty list if something goes wrong


In [12]:
from transformers import pipeline

# Load paraphraser model
paraphraser = pipeline(
    "text2text-generation",
    model="eugenesiow/bart-paraphrase",
    device=0  # or device=-1 if you are on CPU
)


Device set to use cuda:0


In [13]:
import json
import math
from pathlib import Path
from collections import Counter
import torch
from transformers import pipeline

# -------------------------------
# 1. Load filtered training data
# -------------------------------
train_path = Path("..") / "data" / "splits_single_label" / "train.jsonl"

with open(train_path, "r", encoding="utf-8") as f:
    filtered_train = [json.loads(line) for line in f]

filtered_train_labels = [sample["label"] for sample in filtered_train]
train_class_counter = Counter(filtered_train_labels)

# -------------------------------
# 2. Determine augmentation plan
# -------------------------------
target_samples_per_class = 300
minority_classes = [c for c, count in train_class_counter.items() if count < target_samples_per_class]

class_to_paraphrase_per_sample = {
    c: math.ceil((target_samples_per_class - train_class_counter[c]) / train_class_counter[c])
    for c in minority_classes
}

print("\nClasses selected for augmentation:")
for c in minority_classes:
    print(f"Class {c}: {train_class_counter[c]} → {target_samples_per_class} using {class_to_paraphrase_per_sample[c]}x per sample")

# -------------------------------
# 3. Load paraphraser
# -------------------------------
paraphraser = pipeline(
    "text2text-generation",
    model="eugenesiow/bart-paraphrase",
    device=0  # or -1 for CPU
)

def paraphrase_text(text, num_return_sequences, paraphraser_pipeline):
    try:
        outputs = paraphraser_pipeline(
            text,
            num_return_sequences=num_return_sequences,
            num_beams=num_return_sequences + 4,
            max_length=128,
            do_sample=False
        )
        return [o['generated_text'] for o in outputs]
    except Exception as e:
        print(f"[Warning] Skipped due to: {e}")
        return []

# -------------------------------
# 4. Perform augmentation
# -------------------------------
augmented_raw_samples = []

for class_id in minority_classes:
    print(f"\n--- Augmenting Class {class_id} ---")
    original_samples = [s for s in filtered_train if s["label"] == class_id]
    r_c = class_to_paraphrase_per_sample[class_id]

    print(f"Samples available: {len(original_samples)}, generating {r_c} per sample → {len(original_samples) * r_c} total")

    for sample in original_samples:
        raw_text = sample["text"]
        paraphrased = paraphrase_text(raw_text, r_c, paraphraser)

        for p_text in paraphrased:
            augmented_raw_samples.append({
                "text": p_text,
                "label": class_id
            })

print(f"Augmentation complete. Total new augmented samples: {len(augmented_raw_samples)}")



Classes selected for augmentation:
Class 12: 203 → 300 using 1x per sample
Class 23: 88 → 300 using 3x per sample
Class 21: 51 → 300 using 5x per sample
Class 16: 39 → 300 using 7x per sample
Class 19: 85 → 300 using 3x per sample


Device set to use cuda:0



--- Augmenting Class 12 ---
Samples available: 203, generating 1 per sample → 203 total


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



--- Augmenting Class 23 ---
Samples available: 88, generating 3 per sample → 264 total

--- Augmenting Class 21 ---
Samples available: 51, generating 5 per sample → 255 total

--- Augmenting Class 16 ---
Samples available: 39, generating 7 per sample → 273 total

--- Augmenting Class 19 ---
Samples available: 85, generating 3 per sample → 255 total
Augmentation complete. Total new augmented samples: 1250


In [14]:
# View random samples per class
import random
from collections import defaultdict

class_to_augmented = defaultdict(list)

for sample in augmented_raw_samples:
    class_to_augmented[sample['label']].append(sample['text'])

print("\n=== Viewing 8 randomly picked augmented samples per minority class ===")

for class_id in sorted(minority_classes):
    print(f"\n--- Class {class_id} ---")
    samples = random.sample(class_to_augmented[class_id], k=min(8, len(class_to_augmented[class_id])))
    for i, s in enumerate(samples, 1):
        print(f"Sample {i}: {s}")



=== Viewing 8 randomly picked augmented samples per minority class ===

--- Class 12 ---
Sample 1: I'm so uncomfortable.
Sample 2: *people’s ..this is awkward
Sample 3: It's embarrassing to share a license with them, that's for sure.
Sample 4: *concussion* *Trophy system kills it* *Another concussion* * Trophy System kills it * "Yo guys watch me throw this cluster" Honestly embarrassing 
Sample 5: Please tell me you were buying something extremely embarassing!
Sample 6: Nobody said they don't have the right. It's still embarrassing.
Sample 7: [NAME] type awkwardness that some thots go nuts for.
Sample 8: Those women's marches are honestly embarrasing. Most of the people there only seem to be feminists because it's trendy.

--- Class 16 ---
Sample 1: Even today, many years later, that still gives me chills. RIP
Sample 2: Well that’s not what was said he said drugs that kill you quicker than your meant to die, nicotine fits that description.
Sample 3: They really weren't people, they we

#### merge the original + augmented datasets

In [15]:
import json
from pathlib import Path

# -------------------------------
# 1. Load validation and test splits
# -------------------------------
val_path = Path("..") / "data" / "splits_single_label" / "validation.jsonl"
test_path = Path("..") / "data" / "splits_single_label" / "test.jsonl"

def load_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

filtered_validation = load_jsonl(val_path)
filtered_test = load_jsonl(test_path)

# -------------------------------
# 2. Annotate splits and merge
# -------------------------------

# Add split tags
train_all = [{"text": s["text"], "label": s["label"], "split": "train"} for s in filtered_train + augmented_raw_samples]
validation_all = [{"text": s["text"], "label": s["label"], "split": "validation"} for s in filtered_validation]
test_all = [{"text": s["text"], "label": s["label"], "split": "test"} for s in filtered_test]

merged_dataset = train_all + validation_all + test_all

print(f"Merged total: {len(merged_dataset)} (Train: {len(train_all)}, Val: {len(validation_all)}, Test: {len(test_all)})")

# -------------------------------
# 3. Save as JSONL
# -------------------------------
save_path = Path("..") / "data" / "augmented_single_label.jsonl"
save_path.parent.mkdir(parents=True, exist_ok=True)

with open(save_path, "w", encoding="utf-8") as f:
    for sample in merged_dataset:
        json.dump(sample, f, ensure_ascii=False)
        f.write("\n")

print(f"Merged dataset with splits saved to: {save_path.resolve()}")


Merged total: 46696 (Train: 37558, Val: 4548, Test: 4590)
Merged dataset with splits saved to: D:\Documents\FYP\emotion-retrieval-embeddings\data\augmented_single_label.jsonl


### Sampler design

In [16]:
# Define a balanced batch sampler for single-label samples
from torch.utils.data import Sampler
import random
from collections import defaultdict

class UniformBalancedBatchSampler(Sampler):
    def __init__(self, dataset, batch_size, num_classes, labels):
        self.dataset = dataset
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.labels = labels

        # Build mapping: class -> list of indices
        self.class_to_indices = defaultdict(list)
        for idx, label in enumerate(labels):
            self.class_to_indices[label].append(idx)

        self.available_classes = [c for c in range(self.num_classes) if len(self.class_to_indices[c]) > 0]

        if self.batch_size < len(self.available_classes):
            raise ValueError(f"Batch size must be >= number of available classes ({len(self.available_classes)}). Got {self.batch_size}.")

    def __iter__(self):
        while True:
            batch_indices = []

            # Sample one from each available class
            for class_id in self.available_classes:
                candidates = self.class_to_indices[class_id]
                if candidates:
                    selected = random.choice(candidates)
                    batch_indices.append(selected)

            # Fill remaining slots randomly
            while len(batch_indices) < self.batch_size:
                class_id = random.choice(self.available_classes)
                candidates = self.class_to_indices[class_id]
                selected = random.choice(candidates)
                batch_indices.append(selected)

            random.shuffle(batch_indices)
            yield batch_indices

    def __len__(self):
        return 1000000  # or some large number


### Loss function

In [17]:
import torch
import torch.nn.functional as F

class BSCLossSingleLabel(torch.nn.Module):
    def __init__(self, temperature=0.1):
        super(BSCLossSingleLabel, self).__init__()
        self.temperature = temperature

    def forward(self, features, labels):
        """
        Args:
            features: Tensor of shape (batch_size, hidden_dim) - embeddings
            labels: Tensor of shape (batch_size,) - single integer label per sample
        """
        device = features.device
        batch_size = features.shape[0]
        
        # Normalize features to unit hypersphere
        features = F.normalize(features, p=2, dim=1)

        # Compute similarity matrix (batch_size x batch_size)
        sim_matrix = torch.matmul(features, features.T) / self.temperature

        # Positive mask: label[i] == label[j], and i != j
        labels = labels.view(-1, 1)  # shape (batch_size, 1)
        positive_mask = (labels == labels.T).float()  # (batch_size, batch_size)
        diag_mask = torch.eye(batch_size, device=device)
        positive_mask = positive_mask * (1 - diag_mask)  # remove diagonal

        losses = []

        for i in range(batch_size):
            pos_indices = positive_mask[i].nonzero(as_tuple=False).squeeze(1)

            if len(pos_indices) == 0:
                continue

            numerator = torch.exp(sim_matrix[i, pos_indices])

            denominator = 0.0
            for c in torch.unique(labels):
                class_indices = (labels.squeeze() == c).nonzero(as_tuple=False).squeeze(1)
                class_indices = class_indices[class_indices != i]

                if len(class_indices) == 0:
                    continue

                class_sims = torch.exp(sim_matrix[i, class_indices])
                class_sum = class_sims.sum()
                class_sum = class_sum / len(class_indices)

                denominator += class_sum

            loss_i = - torch.mean(torch.log(numerator / denominator))
            losses.append(loss_i)

        if len(losses) == 0:
            return torch.tensor(0.0, device=device)

        return torch.mean(torch.stack(losses))


#### test my loss function

In [18]:
import torch
import torch.nn.functional as F
import numpy as np

def print_sim_matrix_info(features, labels, temperature=0.1):
    print("\n=== Detailed BSCL Diagnostic ===")
    features = F.normalize(features, p=2, dim=1)
    sim_matrix = torch.matmul(features, features.T) / temperature

    print("\nSimilarity Matrix:")
    print(sim_matrix)

    labels = labels.view(-1, 1)
    batch_size = labels.size(0)
    pos_mask = (labels == labels.T).float() - torch.eye(batch_size)
    print("\nPositive Mask (1 = same class, 0 = other):")
    print(pos_mask)

    print("\nClass-wise indices:")
    for c in torch.unique(labels):
        indices = (labels.squeeze() == c).nonzero(as_tuple=False).squeeze(1)
        print(f"Class {c.item()}: indices {indices.tolist()}")

    return sim_matrix, pos_mask

features = F.normalize(torch.randn(8, 5), dim=1)  # Random 8x5 features
labels = torch.tensor([0, 0, 1, 1, 2, 2, 3, 3])   # 4 classes, 2 samples each

sim_matrix, pos_mask = print_sim_matrix_info(features, labels, temperature=0.1)



=== Detailed BSCL Diagnostic ===

Similarity Matrix:
tensor([[10.0000, -1.2799, -4.4199,  7.7487, -3.4071, -0.1243, -8.3086,  7.1075],
        [-1.2799, 10.0000,  1.6897,  0.6522, -2.5841,  1.6311,  5.5117, -3.7046],
        [-4.4199,  1.6897, 10.0000, -6.9804, -6.4275, -6.7476,  3.7009, -0.7663],
        [ 7.7487,  0.6522, -6.9804, 10.0000,  0.5539,  5.7648, -4.0981,  6.2141],
        [-3.4071, -2.5841, -6.4275,  0.5539, 10.0000,  5.6240,  1.7582, -4.3310],
        [-0.1243,  1.6311, -6.7476,  5.7648,  5.6240, 10.0000,  3.1521,  0.3289],
        [-8.3086,  5.5117,  3.7009, -4.0981,  1.7582,  3.1521, 10.0000, -5.4888],
        [ 7.1075, -3.7046, -0.7663,  6.2141, -4.3310,  0.3289, -5.4888, 10.0000]])

Positive Mask (1 = same class, 0 = other):
tensor([[0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.

In [19]:
import torch
import torch.nn.functional as F

# Define a dummy 8-vector batch with 4 classes (2 per class)
x = torch.tensor([
    [1.0, 0.0, 0.0],    # Class 0
    [0.9, 0.1, 0.0],    # Class 0
    [0.0, 1.0, 0.0],    # Class 1
    [0.0, 0.9, 0.1],    # Class 1
    [0.0, 0.0, 1.0],    # Class 2
    [0.1, 0.0, 0.9],    # Class 2
    [-1.0, 0.0, 0.0],   # Class 3
    [-0.9, -0.1, 0.0],  # Class 3
])
y = torch.tensor([0, 0, 1, 1, 2, 2, 3, 3])

# Normalize
x_norm = F.normalize(x, p=2, dim=1)
sim_matrix = torch.matmul(x_norm, x_norm.T) / 0.1

# Compute positive mask
labels = y.view(-1, 1)
positive_mask = (labels == labels.T).float()
diag = torch.eye(len(y))
positive_mask = positive_mask * (1 - diag)

# Print diagnostics
print("=== Similarity Matrix ===")
print(sim_matrix.round(decimals=2))  # FIXED: use decimals=2 instead of positional arg

print("\n=== Positive Mask ===")
print(positive_mask)

print("\n=== Positive Indices per Sample ===")
for i in range(len(y)):
    pos_indices = (positive_mask[i] > 0).nonzero(as_tuple=False).squeeze(1).tolist()
    print(f"Sample {i}: Positives -> {pos_indices}, Class: {y[i].item()}")


=== Similarity Matrix ===
tensor([[ 10.0000,   9.9400,   0.0000,   0.0000,   0.0000,   1.1000, -10.0000,
          -9.9400],
        [  9.9400,  10.0000,   1.1000,   1.1000,   0.0000,   1.1000,  -9.9400,
         -10.0000],
        [  0.0000,   1.1000,  10.0000,   9.9400,   0.0000,   0.0000,   0.0000,
          -1.1000],
        [  0.0000,   1.1000,   9.9400,  10.0000,   1.1000,   1.1000,   0.0000,
          -1.1000],
        [  0.0000,   0.0000,   0.0000,   1.1000,  10.0000,   9.9400,   0.0000,
           0.0000],
        [  1.1000,   1.1000,   0.0000,   1.1000,   9.9400,  10.0000,  -1.1000,
          -1.1000],
        [-10.0000,  -9.9400,   0.0000,   0.0000,   0.0000,  -1.1000,  10.0000,
           9.9400],
        [ -9.9400, -10.0000,  -1.1000,  -1.1000,   0.0000,  -1.1000,   9.9400,
          10.0000]])

=== Positive Mask ===
tensor([[0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.,

In [20]:

import torch
import torch.nn.functional as F
from torch import nn

# Simulate realistic batch of features from a small model
torch.manual_seed(42)
batch_size = 32
feature_dim = 64
num_classes = 8

# Randomly generate normalized feature vectors
features = F.normalize(torch.randn(batch_size, feature_dim), p=2, dim=1)

# Randomly assign labels (integers from 0 to num_classes-1)
labels = torch.randint(low=0, high=num_classes, size=(batch_size,))

# Create criterion and compute loss
criterion = BSCLossSingleLabel(temperature=0.1)
loss_value = criterion(features, labels)

loss_value.item()

2.9187521934509277

### Trainning plan
1.  Stage 1 creates a strong "backbone" embedding space, where emotions are cleanly separated. Filter GoEmotions to only single-label sentences
2. Stage 2 teaches the model the complex, messy mixtures that real human feelings exhibit


In [21]:
from datasets import load_dataset
from pathlib import Path

# Correct path to the actual file
dataset_path = Path("..") / "data" / "augmented_single_label.jsonl"

dataset = load_dataset("json", data_files=str(dataset_path), split="train")





Generating train split: 0 examples [00:00, ? examples/s]